In [1]:
import sys
sys.path.append('..')

In [2]:
from curry.loader import Loader
from curry.clean import Cleaner
from curry.features import Extractor

In [3]:
loader = Loader()
extractor = Extractor('tf')

In [4]:
extractor.keywords(loader.sublessons_w_content().content)

cleaning:  35%|██████████████████████████████▌                                                         | 4051/11667 [00:20<00:34, 218.02it/s]/home/rsebastian/.anaconda3/envs/curry/lib/python3.7/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
keyword ex.: 100%|████████████████████████████████████████████████████████████████████████████████████| 11667/11667 [00:53<00:00, 218.44it/s]


<11667x7684 sparse matrix of type '<class 'numpy.int64'>'
	with 229090 stored elements in Compressed Sparse Row format>

In [ ]:
extractor

In [7]:
cleaner.clean(df.content.iloc[0])

'Bei den Aufgaben zur Dichte lässt sich das Volumen mancher (sehr einfacher) Körper rechnerisch ermitteln.  Im Folgenden sind die Formeln für einige wichtige Umfangs-, Flächen- und Volumenberechnungen angegeben'

In [17]:
import pandas as pd
import pickle

In [9]:
lessons = pd.read_csv('../data/lessons.csv', index_col=0)

In [12]:
detail_pages = pd.read_csv('../data/detail_pages.csv', index_col=0)

In [21]:
sublessons = pd.merge(
        lessons, 
        detail_pages, 
        left_on='url', right_on='lesson_url').drop(columns='lesson_url')

In [22]:
sublessons.head(2)

,land,klass,name,url,lessons_cat,grundwissen_url
0,baden-wuerttemberg,5,"Masse, Volumen und Dichte",/mechanik/masse-volumen-und-dichte,mechanik,/mechanik/masse-volumen-und-dichte/grundwissen...
1,baden-wuerttemberg,5,"Masse, Volumen und Dichte",/mechanik/masse-volumen-und-dichte,mechanik,/mechanik/masse-volumen-und-dichte/grundwissen...


In [28]:
import os

In [29]:
len(os.listdir('../data/grundwissen_pages/'))

556

In [24]:


def to_kws(content):
    ranked_kws = yake.KeywordExtractor(lan='de').extract_keywords(content)
    sorted_kws = sorted(ranked_kws, key=lambda x: x[1], reverse=True)
    return [kw for (kw, _) in sorted_kws]


In [27]:
sublessons.grundwissen_url.apply(read_file)

0        <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
1        <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
2        <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
3        <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
4        <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
                               ...                        
11685    <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
11686    <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
11687    <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
11688    <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
11689    <!DOCTYPE html>\n<html lang="de" dir="ltr">\n<...
Name: grundwissen_url, Length: 11690, dtype: object

In [ ]:
!ls ../data/grundwissen_pages/SLASHmechanikSLASHmasse-volumen-und-dicht

In [18]:
with open('../data/keywords.pkl', 'rb') as f:
    kw = pickle.load(f)

ValueError: unsupported pickle protocol: 5